In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad

# Constants
G = 6.67e-11  # m^3/kg/s^2
g = 9.81  # m/s^2
M_earth = 5.9e24  # kg
M_moon = 7.3e22  # kg
R_earth = 6378e3  # m
R_moon = 1737e3  # m
d_earth_moon = 3.8e8  # m
m_cmd = 5500  # kg
ve = 2.4e3  # m/s
mdot = 1.3e4  # kg/s
m0 = 2.8e6  # kg
mf = 7.5e5  # kg

# Part 1: Gravitational Potential
def gravitational_potential(M, xM, yM, x, y):
    r = np.sqrt((x - xM)**2 + (y - yM)**2)
    r = np.maximum(r, 1e3)  # avoid singularity
    return -G * M / r

# 1D plot
x_vals = np.linspace(R_earth, 1.5 * d_earth_moon, 1000)
phi_vals = np.abs(gravitational_potential(M_earth, 0, 0, x_vals, 0))

plt.figure()
plt.plot(x_vals / 1e6, phi_vals)
plt.yscale("log")
plt.xlabel("Distance from Earth center (Mm)")
plt.ylabel("|Gravitational Potential| (J/kg)")
plt.title("Gravitational Potential vs Distance")
plt.grid()
plt.show()

# 2D color mesh of Earth's potential
grid_pts = 500
x = np.linspace(-1.5 * d_earth_moon, 1.5 * d_earth_moon, grid_pts)
y = np.linspace(-1.5 * d_earth_moon, 1.5 * d_earth_moon, grid_pts)
X, Y = np.meshgrid(x, y)
phi = gravitational_potential(M_earth, 0, 0, X, Y)

plt.figure()
plt.pcolormesh(X / 1e6, Y / 1e6, np.abs(phi), shading='auto', norm='log')
plt.colorbar(label='|Potential| (J/kg)')
plt.title("Gravitational Potential of Earth")
plt.axis("equal")
plt.xlabel("x (Mm)")
plt.ylabel("y (Mm)")
plt.show()

# Part 2: Earth-Moon System Potential
x_moon = d_earth_moon / np.sqrt(2)
y_moon = d_earth_moon / np.sqrt(2)
phi_total = gravitational_potential(M_earth, 0, 0, X, Y) + gravitational_potential(M_moon, x_moon, y_moon, X, Y)

plt.figure()
plt.pcolormesh(X / 1e6, Y / 1e6, np.abs(phi_total), shading='auto', norm='log')
plt.colorbar(label='|Potential| (J/kg)')
plt.title("Gravitational Potential: Earth + Moon")
plt.axis("equal")
plt.xlabel("x (Mm)")
plt.ylabel("y (Mm)")
plt.show()

# Contour plot
plt.figure()
contours = plt.contour(X / 1e6, Y / 1e6, np.log10(np.abs(phi_total)), levels=20)
plt.clabel(contours, inline=True)
plt.title("Contour of log|Gravitational Potential|")
plt.xlabel("x (Mm)")
plt.ylabel("y (Mm)")
plt.axis("equal")
plt.show()

# Part 3: Gravitational Force
def gravitational_force(M1, m2, x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    r2 = dx**2 + dy**2
    r2 = np.maximum(r2, 1e6)  # avoid divide-by-zero
    r = np.sqrt(r2)
    F = -G * M1 * m2 / r2
    return F * dx / r, F * dy / r

# Vector field of forces
Fx_total = np.zeros_like(X)
Fy_total = np.zeros_like(Y)

for i in range(grid_pts):
    for j in range(grid_pts):
        x_pos, y_pos = X[i, j], Y[i, j]
        Fx1, Fy1 = gravitational_force(M_earth, m_cmd, 0, 0, x_pos, y_pos)
        Fx2, Fy2 = gravitational_force(M_moon, m_cmd, x_moon, y_moon, x_pos, y_pos)
        Fx_total[i, j] = Fx1 + Fx2
        Fy_total[i, j] = Fy1 + Fy2

magnitude = np.sqrt(Fx_total**2 + Fy_total**2)

plt.figure()
plt.streamplot(x / 1e6, y / 1e6, Fx_total.T, Fy_total.T, color=np.log10(magnitude.T), linewidth=1)
plt.colorbar(label='log10(|Force|) [N]')
plt.title("Gravitational Force Field: Earth + Moon on Apollo")
plt.xlabel("x (Mm)")
plt.ylabel("y (Mm)")
plt.axis("equal")
plt.show()

# Part 4: Rocket Altitude
def delta_v(t, m0, mf, mdot, ve, g):
    m_t = np.maximum(m0 - mdot * t, mf)
    if m_t > mf:
        return ve * np.log(m0 / m_t) - g * t
    else:
        return 0

T = (m0 - mf) / mdot
print(f"Burn time T: {T:.2f} s")

v_func = lambda t: delta_v(t, m0, mf, mdot, ve, g)
h, _ = quad(v_func, 0, T)
print(f"Rocket altitude at burnout: {h:.2f} m")
